In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
from datetime import datetime

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the API key.
import requests
from config import weather_api_key
from config import g_key

In [4]:
city_weather_df = pd.read_csv("data/WeatherPy_database.csv")
city_weather_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain (inches),Snow (inches)
0,0,Vaini,TO,-21.20,-175.20,69.8,88,75,13.87,broken clouds,0.0,0
1,1,Talnakh,RU,69.49,88.40,55.4,87,62,8.95,broken clouds,0.0,0
2,2,Severo-Yeniseyskiy,RU,60.37,93.04,59.2,75,100,4.74,overcast clouds,0.0,0
3,3,Nikolskoye,RU,59.70,30.79,64.0,72,75,2.75,light intensity shower rain,0.0,0
4,4,Punta Arenas,CL,-53.15,-70.92,39.0,78,20,21.92,few clouds,0.0,0


In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_choice = input("Do you want it to be raining? (yes/no) ")
snow_choice = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [6]:
city_weather_df.loc[city_weather_df['Rain (inches)'] > 0, 'Raining'] = 'yes'
city_weather_df.loc[city_weather_df['Rain (inches)'] == 0, 'Raining'] = 'no'
city_weather_df.loc[city_weather_df['Snow (inches)'] > 0, 'Snowing'] = 'yes'
city_weather_df.loc[city_weather_df['Snow (inches)'] == 0, 'Snowing'] = 'no'
city_weather_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain (inches),Snow (inches),Raining,Snowing
0,0,Vaini,TO,-21.20,-175.20,69.8,88,75,13.87,broken clouds,0.0,0,no,no
1,1,Talnakh,RU,69.49,88.40,55.4,87,62,8.95,broken clouds,0.0,0,no,no
2,2,Severo-Yeniseyskiy,RU,60.37,93.04,59.2,75,100,4.74,overcast clouds,0.0,0,no,no
3,3,Nikolskoye,RU,59.70,30.79,64.0,72,75,2.75,light intensity shower rain,0.0,0,no,no
4,4,Punta Arenas,CL,-53.15,-70.92,39.0,78,20,21.92,few clouds,0.0,0,no,no


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) & \
                                       (city_weather_df["Max Temp"] >= min_temp) & \
                                       (city_weather_df["Raining"] == rain_choice) & \
                                       (city_weather_df["Snowing"] == snow_choice)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain (inches),Snow (inches),Raining,Snowing
22,22,Kabo,CF,7.70,18.63,75.02,85,43,0.31,light rain,0.26,0,yes,no
42,42,Kaura Namoda,NG,12.59,6.58,75.97,66,98,19.95,light rain,0.91,0,yes,no
77,77,Amahai,ID,-3.33,128.92,77.02,88,99,5.32,moderate rain,1.47,0,yes,no
80,80,Gasan,PH,13.32,121.85,80.01,89,100,3.91,moderate rain,1.67,0,yes,no
109,109,Kirakira,SB,-10.45,161.92,78.96,84,82,13.44,light rain,0.56,0,yes,no
111,111,Bac Giang,VN,21.27,106.20,86.00,62,92,5.82,light rain,0.30,0,yes,no
112,112,Butaritari,KI,3.07,172.79,82.69,79,53,15.70,light rain,0.13,0,yes,no
127,127,Buala,SB,-8.14,159.59,80.33,83,93,11.79,moderate rain,1.27,0,yes,no
146,146,Pandan,PH,14.05,124.17,77.63,89,100,10.13,moderate rain,2.03,0,yes,no
156,156,Yuli,NG,9.70,10.27,76.21,85,100,2.24,light rain,0.34,0,yes,no


In [9]:
preferred_cities_df.count()

Unnamed: 0             43
City                   43
Country                43
Lat                    43
Lng                    43
Max Temp               43
Humidity               43
Cloudiness             43
Wind Speed             43
Weather Description    43
Rain (inches)          43
Snow (inches)          43
Raining                43
Snowing                43
dtype: int64

In [9]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

NameError: name 'preferred_cities_df' is not defined

In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [2]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

NameError: name 'hotel_df' is not defined

In [24]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
22,Kabo,CF,75.02,light rain,7.70,18.63,
42,Kaura Namoda,NG,75.97,light rain,12.59,6.58,Moonlight Hotel
77,Amahai,ID,77.02,moderate rain,-3.33,128.92,Hotel isabella
80,Gasan,PH,80.01,moderate rain,13.32,121.85,JADE FRANCISCO
109,Kirakira,SB,78.96,light rain,-10.45,161.92,SanBiz Lodge


In [25]:
hotel_df.to_csv('WeatherPy_vacation.csv')

In [1]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Call the figure to plot the data.

fig

NameError: name 'hotel_df' is not defined